# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [60]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import gmaps

# Import API key
from api_keys import g_api_key

In [61]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,saint-philippe,-21.3585,55.7679,81.99,75,91,9.08,RE,1680412909
1,1,jiuquan,39.7432,98.5174,42.26,24,12,12.68,CN,1680412994
2,2,hithadhoo,-0.6000,73.0833,82.13,72,66,8.37,MV,1680412923
3,3,gat,31.6100,34.7642,59.81,100,0,3.20,IL,1680413001
4,4,kloulklubed,7.0419,134.2556,82.90,72,78,4.45,PW,1680413220


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [62]:
%%capture --no-display

# Configure the map plot
city_map = city_data_df.hvplot.points("Lng", "Lat", geo = True, color = "City", alpha = 0.6, size = "Humidity",
                                      tiles = "OSM"
                                     )

# Display the map
city_map

ImportError: Geographic projection support requires geoviews, pyproj and cartopy.

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [63]:
# Narrow down cities that fit criteria and drop any results with null values
# YOUR CODE HERE
filtered_df = city_data_df
filtered_df = filtered_df.loc[(filtered_df["Max Temp"] < 80.6) & (filtered_df["Max Temp"] > 69.8)]
filtered_df = filtered_df.loc[filtered_df["Wind Speed"] < 10]
filtered_df = filtered_df.loc[filtered_df["Cloudiness"] == 0]

# Drop any rows with null values
# YOUR CODE HERE
filtered_df = filtered_df.dropna()
# Display sample data
# YOUR CODE HERE
filtered_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
85,85,acapulco,16.8634,-99.8901,78.62,78,0,0.00,MX,1680412963
115,115,kapaa,22.0752,-159.3190,71.92,84,0,4.61,US,1680412916
137,137,maceio,-9.6658,-35.7353,74.64,94,0,4.61,BR,1680412681
255,255,sao filipe,14.8961,-24.4956,78.76,47,0,6.80,CV,1680412741
356,356,seminole,28.7086,-81.2081,74.41,89,0,8.05,US,1680413287
398,398,shikarpur,27.9571,68.6379,80.49,34,0,9.91,PK,1680413295
485,485,inhambane,-23.8650,35.3833,73.38,94,0,6.24,MZ,1680413309
504,504,maxixe,-23.8597,35.3472,73.22,94,0,5.19,MZ,1680413313
525,525,havelock,34.8791,-76.9013,69.96,83,0,9.22,US,1680412971


### Step 3: Create a new DataFrame called `hotel_df`.

In [64]:
hotel_df = filtered_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
85,acapulco,MX,16.8634,-99.8901,78,
115,kapaa,US,22.0752,-159.3190,84,
137,maceio,BR,-9.6658,-35.7353,94,
255,sao filipe,CV,14.8961,-24.4956,47,
356,seminole,US,28.7086,-81.2081,89,
398,shikarpur,PK,27.9571,68.6379,34,
485,inhambane,MZ,-23.8650,35.3833,94,
504,maxixe,MZ,-23.8597,35.3472,94,
525,havelock,US,34.8791,-76.9013,83,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [65]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "apiKey":g_api_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    # YOUR CODE HERE
    longitude = row["Lng"]
    latitude = row["Lat"]
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params = params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
acapulco - nearest hotel: Hotel del Valle
kapaa - nearest hotel: Pono Kai Resort
maceio - nearest hotel: Maceió Hostel e Pousada
sao filipe - nearest hotel: Bela Vista
seminole - nearest hotel: Best Western Plus Sanford
shikarpur - nearest hotel: No hotel found
inhambane - nearest hotel: Hotel de Inhambane
maxixe - nearest hotel: Hotel Herry
havelock - nearest hotel: Quality Inn Havelock Hwy 70


,City,Country,Lat,Lng,Humidity,Hotel Name
85,acapulco,MX,16.8634,-99.8901,78,Hotel del Valle
115,kapaa,US,22.0752,-159.3190,84,Pono Kai Resort
137,maceio,BR,-9.6658,-35.7353,94,Maceió Hostel e Pousada
255,sao filipe,CV,14.8961,-24.4956,47,Bela Vista
356,seminole,US,28.7086,-81.2081,89,Best Western Plus Sanford
398,shikarpur,PK,27.9571,68.6379,34,No hotel found
485,inhambane,MZ,-23.8650,35.3833,94,Hotel de Inhambane
504,maxixe,MZ,-23.8597,35.3472,94,Hotel Herry
525,havelock,US,34.8791,-76.9013,83,Quality Inn Havelock Hwy 70


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [66]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE
hotel_map = hotel_df.hvplot.points("Lng", 
                                   "Lat", 
                                   geo = True,
                                   color = "City",
                                   alpha = 0.8,
                                   size = "Humidity",
                                   tiles = "OSM",
                                   hover_cols = ["Hotel Name", "Country"]
                                  )
# Display the map
# YOUR CODE HERE
hotel_map

ImportError: Geographic projection support requires geoviews, pyproj and cartopy.